##### Trial and error for dashboard 
__Larissa Voshol__  
__11/12/2023__  
__BHVF2__  

In [1]:
import numpy as np
import pandas as pd
import panel as pn
import bokeh as bk
import holoviews as hv
import param

pn.extension()


In [2]:
#import data
filepath_ct = "../OPC_CT_radiomics_TCIA.csv"
filepath_data = "../OPC_data_clinical.csv"

#make dataframes good
df_ct = pd.read_csv(filepath_ct)
del df_ct[df_ct.columns[0]]
df_ct.set_index("patient", inplace=True)
df_data = pd.read_csv(filepath_data)
df_data.set_index("patient", inplace=True)


In [3]:
basic_information = 0 
shape_features = 23
first_order_features = 38
GLCM_features = 55 
GLDM_features = 79
GLRLM_features = 93 
GLZSM_features = 109 
NGTDM_features = 125

start website

In [5]:
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, linear_cmap
df_shape = df_ct.iloc[:,shape_features:first_order_features-1]

def render_plot(x, y, z, title):
    #grap group column
    group = df_data.loc[:,z]
    #paste to the frame with the data for the plot
    df = pd.concat([df_shape, group], axis=1)
    #unique factors
    uniq = df[z].unique()

    if len(uniq) == 1:
        color = "purple"
    if len(uniq) <= 2:
        color = factor_cmap(z, ["#1f77b4", "#aec7e8"], uniq)
    if len(uniq) <=20:
        color = factor_cmap(z, bk.palettes.Category20[len(uniq)], uniq)
    else:
        z=""
        color=""

    if y == False:
        y = range(len(df)) 

    p = figure(title = title)
    # p.xaxis.axis_label = x
    # p.yaxis.axis_label = y

    p.scatter(x, y, source=df, color=color, legend_field=z)
    
    return p

In [6]:
# from bokeh.plotting import figure, show
# from bokeh.transform import factor_cmap
# from bokeh.models import ColumnDataSource, CDSView, GroupFilter
# df_shape = df_ct.iloc[:,shape_features:first_order_features-1]


# shape_cds = ColumnDataSource(df_shape)

# def render_plot(x, y, z, plot_title):
    


In [7]:
def click_graph_btn(event):
    if not event:
        return    
    body.append(pn.layout.Divider()) 
    body.append(pn.Row(
        pn.Column("# Make a graph", make_graph_input, graph_btn, pn.bind(click_graph_btn, graph_btn)),
        pn.Card(pn.bind(render_plot, x=select_feature1, y=select_feature2, z=select_group, title=input_title), title="Graph")
    ))

    
    

In [8]:
def change_layout():
    make_graph_input =  pn.Card(select_feature1, select_group, input_title, title="Select your features")
    
    figsingle = pn.bind(render_plot, x=select_feature1, y=False, z=select_group, title=input_title, figsingle=True)
    dashboard.main.append(
     pn.Row(
        pn.Column("# Make a graph", make_graph_input, graph_btn),
        pn.Card(figsingle, title="Graph"))
    )
    

In [9]:
#options for group
df_nonan = df_data.dropna(axis=1)

#select variables for graph
select_feature1 = pn.widgets.Select(name="Variable for x-axis", options=list(df_shape))
select_feature2 = pn.widgets.Select(name="Variable for y-axis", options=list(df_shape))
select_group = pn.widgets.Select(name="Based on wich group", options=list(df_nonan))
input_title = pn.widgets.TextInput(name="Graph title", placeholder="Enter a string here...")
graph_btn = pn.widgets.Button(name="Make new Graph", button_type="primary")
change_graph_switch = pn.widgets.Switch(name="1 or 2 variables")

#click on buttons
graph_btn.on_click(click_graph_btn)

#switches
change_graph_switch.controls(
    dashboard.main.append(
     pn.Column(pn.Row(
        pn.Column("# Make a graph", pn.Card(select_feature1, select_group, input_title, title="Select your features"), graph_btn),
        pn.Card(pn.bind(render_plot, x=select_feature1, y=False, z=select_group, title=input_title), title="Graph"))
    )
    ))
change_graph_switch.tooltip("### switch between 1 or 2 variables")



#bind objects
figdouble = pn.bind(render_plot, x=select_feature1, y=select_feature2, z=select_group, title=input_title)

#layout for main
make_graph_input =  pn.Card(select_feature1, select_feature2, select_group, input_title, title="Select your features")
body = pn.Column(pn.Row(
        pn.Column(change_graph_switch),
        pn.Column("# Make a graph", make_graph_input, graph_btn),
        pn.Card(figdouble, title="Graph"))
    )

#Instantiate the template with widgets displayed in the sidebar
dashboard = pn.template.BootstrapTemplate(
    title="PlotRadionomics",
    sidebar=[]
)

# Append a layout to the main area, to demonstrate the list-like API
dashboard.main.append(body)

dashboard.show();


NameError: name 'dashboard' is not defined

voor de show

In [ ]:
#options for group
df_nonan = df_data.dropna(axis=1)

#select variables for graph

select_feature1 = pn.widgets.Select(name="Variable for x-axis", options=list(df_shape))
select_feature2 = pn.widgets.Select(name="Variable for y-axis", options=list(df_shape))
select_group = pn.widgets.Select(name="Based on wich group", options=list(df_nonan))
input_title = pn.widgets.TextInput(name="Graph title", placeholder="Enter a string here...")
graph_btn = pn.widgets.Button(name="Make new Graph", button_type="primary")

#click on buttons
graph_btn.param.watch(click_graph_btn, "value")

#bind objects
fig = pn.bind(render_plot, x=select_feature1, y=select_feature2, z=select_group, title=input_title)


#layout for main
make_graph_input =  pn.Card(select_feature1, select_feature2, select_group, input_title, title="Select your features")
body = pn.Row(
        pn.Column("# Make a graph", make_graph_input, graph_btn),
        pn.Column(fig, title="Graph")
    )

#Instantiate the template with widgets displayed in the sidebar
dashboard = pn.template.BootstrapTemplate(
    title="PlotRadionomics",
    sidebar=[]
)

# Append a layout to the main area, to demonstrate the list-like API
dashboard.main.append(pn.Column(body))

dashboard.show();


In [ ]:
def click_switch(value):
    if value == "moi":
        return pn.Row(pn.Column("#meui", graph_btn))
    if value == "meuk":
        return pn.Row(pn.Column("#moi#moi", change_graph_switch, graph_btn))

In [25]:
this = df_nonan["Dose (gy)"]
print(set(this))

{64.0, 66.0, 68.0, 70.0, 74.0, 54.4, 60.0, 62.0}


2024-01-18 13:35:16,608 ERROR: panel.reactive - Callback failed for object named "Based on wich group" changing property {'value': 'Number of fractions'} 
Traceback (most recent call last):
  File "c:\Users\Laris\Documents\repos\p_2.2\DD-radiomics\.venv\Lib\site-packages\panel\reactive.py", line 385, in _process_events
    self.param.update(**self_events)
  File "c:\Users\Laris\Documents\repos\p_2.2\DD-radiomics\.venv\Lib\site-packages\param\parameterized.py", line 2282, in update
    restore = self_._update(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Laris\Documents\repos\p_2.2\DD-radiomics\.venv\Lib\site-packages\param\parameterized.py", line 2322, in _update
    self_._batch_call_watchers()
  File "c:\Users\Laris\Documents\repos\p_2.2\DD-radiomics\.venv\Lib\site-packages\param\parameterized.py", line 2506, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "c:\Users\Laris\Documents\repos\p_2.2\DD-radiomics\.venv\Lib\site-pac

currently working in this

In [621]:
#imports
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, linear_cmap
from bokeh.models import TapTool, HoverTool
from bokeh.models import ColumnDataSource, Whisker

#Edit dataset


#main dataset
temp_df = pd.DataFrame()

#options for group choises
df_radionmics = []
options = []
options_group = []

#Functions

#generate scatter plot with 1 feature
def create_scater_plot(z, title,  df_clini, df_radionmics):
    if z == "options":
        return
    #grab group column
    group = df_clini.loc[:,z]
    #unique factors
    uniq = group.unique()

    if z not in df_radionmics: 
        #paste to the frame with the data for the plot
        group = [str(item) for item in group]
        df_radionmics.insert(0, z, group)
    #order data ascending
    df = df_radionmics.sort_values(by=[z], ascending=True)

    # Create figure object
    TOOLS = "reset,tap,save"
    p = figure(title = title, tools=TOOLS)

    #create colormap for different lengths
    if len(uniq) == 1:
        color = "#1f77b4"
    elif len(uniq) <= 2:
        uniq = [str(item) for item in uniq]
        color = factor_cmap(z, ["#1f77b4", "#aec7e8"], uniq)
    elif len(uniq) <= 20:
        uniq = [str(item) for item in uniq]
        color = factor_cmap(z, bk.palettes.Category20[len(uniq)], uniq)
    elif len(uniq) > 20:
        # create how big the bins will be if you take 20 colors
        max_value = int(uniq.max())
        min_value = int(uniq.min())
        step = max(1, 3*int(round((max_value - min_value)/10, 0)))
        bins = list(range(min_value, max_value + step, step))
        # create the groups and append to dataframe
        new_z = []
        count=0
        for x in group:
            count +=1
            count_value = 0  
            for value in bins[:-1]:
                if bins[count_value] >= float(x) and float(x) < bins[count_value + 1]:
                    new_z.append(bins[count_value])
                    break  
                count_value += 1
        
        # add to datafram by removing the old first
        df.drop([z], axis=1 ,inplace=True)
        df.insert(0, z, new_z)
        df = df.sort_values(by=[z], ascending=True)
        #make colormap
        color = linear_cmap(z, bk.palettes.Category20[10], low=min(bins), high=max(bins))

    return p, color, df

def create_plot_heat( x, all, title, df_radionmics):
    #grab the correct collums according to the widget
    temp_df = {}
    for element in x:
        temp = df_radionmics[element]
        temp_df[element] = temp
    df = pd.DataFrame(data = temp_df)
 
    if all: 
        df = df_radionmics
    
    # Create the correlation df
    df_hm = df.corr()

    # Make the plot
    p = df_hm.hvplot.heatmap(title= title, 
                    colorbar=False, 
                    cmap="fire_r", 
                    xlabel="Feature 1",
                    ylabel= "Feature 2",
                    height = 700,
                    width = 700)
    p.opts(xrotation = 45,
        tools=["hover"])

    return p

def create_plot_f1(x, z, title,  df_clini, df_radionmics):
    #create figure and prep df and colorpallete 
    p, color, df = create_scater_plot(z, title,  df_clini, df_radionmics)

    p.scatter("patient", x, source=df, color=color, legend_field=z)
    hover = HoverTool(tooltips=[("Patient ID", "@patient"),
    ("y-waarde","$x")])
    p.add_tools(hover)
    return p

def create_plot_f2(x, y, z, title, df_clini, df_radionmics):
    #create figure and prep df and colorpallete 
    p, color, df = create_scater_plot(z, title,  df_clini, df_radionmics)
    p.scatter(x, y, source=df, color=color, legend_field=z)
    hover = HoverTool(tooltips=[("Patient ID", "@patient"),
    ("y-waarde","$x"),
    ("x-waarde","$x")])
    p.add_tools(hover)
    return p

def create_boxplot(df_shape, features):
    #grab the correct collums according to the widget
    temp_df = {}
    print(df_shape)
    for element in features:
        print(element)
        temp = df_shape[element]
        print(temp)
        temp_df[element] = temp
    print(temp_df)
    df_shape = pd.DataFrame(data = temp_df)
    #unique keys
    kinds = list(df_shape.columns)

    #create quantile
    qs = df_shape.quantile([0.25, 0.5, 0.75])
    qs.insert(0,"new_index", ["q1", "q2", "q3"])
    temp_qs = qs.set_index("new_index")


    #create lower and upper
    upper=["upper"]
    lower=["lower"]
    for column in temp_qs:
        q1 = temp_qs.loc["q1",column]
        q3 = temp_qs.loc["q3",column]
        iqr = q3 - q1
        upper.append(q3 + 1.5*iqr)
        lower.append(q1 - 1.5*iqr)
    print(df_shape)    
    df_shape.reset_index(inplace=True)
    print(df_shape)
    melt = pd.melt(df_shape, id_vars=['patient'], value_vars=df_shape.columns)


    #add to df
    qs.loc[len(qs.index)] = upper
    qs.loc[len(qs.index)] = lower
    qs.set_index("new_index", inplace=True)

    #switch columns with index
    New_df=qs.T.groupby(level=0).agg(lambda x : x.values.tolist()).stack().apply(pd.Series).unstack().sort_index(level=1,axis=1)
    New_df.columns=New_df.columns.droplevel(level=0)
    New_df.reset_index(inplace=True)

    #merge the quantile etc with the data
    df_boxplot = pd.merge(melt, New_df, left_on="variable", right_on="index", how="right").drop("index", axis=1)

    source = ColumnDataSource(df_boxplot)

    p = figure(x_range=kinds, tools="hover",
            title="Distribution with each feature",
            background_fill_color="#eaefef", y_axis_label="The unit of the feature")

    # outlier range
    whisker = Whisker(base="variable", upper="upper", lower="lower", source=source)
    whisker.upper_head.size = whisker.lower_head.size = 20
    p.add_layout(whisker)

    # quantile boxes
    cmap = factor_cmap("variable", "TolRainbow7", kinds)
    p.vbar("variable", 0.7, "q2", "q3", source=source, color=cmap, line_color="black")
    p.vbar("variable", 0.7, "q1", "q2", source=source, color=cmap, line_color="black")

    # outliers
    outliers = df_boxplot[~df_boxplot.value.between(df_boxplot.lower, df_boxplot.upper)]
    p.scatter("variable", "value", source=outliers, size=6, color="black", alpha=0.3)

    p.xgrid.grid_line_color = None
    p.axis.major_label_text_font_size="14px"
    p.axis.axis_label_text_font_size="12px"
    p.axis.major_label_orientation = 45

    return p


def add_graph_to_layout(items, event):
    add_graphbox = items[0]
    select_graphtype = items[1]
    graph_layout = items[2]
    if not event:
        return
    # make invisible
    if add_graphbox.visible:
        add_graphbox.visible = False
    else:
        #make visibele
        select_graphtype.value = "Options"
        select_graphtype.disabled_options = reset_disabledoptions(graph_layout)
        add_graphbox.visible = True

def reset_disabledoptions(graph_layout):
    disable = []
    for graph in graph_layout:
        if graph == "scatter with 1 feature":
            disable.append("Scatter(1f)")
        elif graph == "scatter with 2 features":
            disable.append("Scatter(2f)")
        elif graph == "Heatmap":
            disable.append("Heatmap")
        elif graph == "Boxplot":
            disable.append("Boxplot")
    return disable
            


def graphtype_disabledoptions(select_graphtype):
    #grab the already disabled graph types
    disablegraphs = []
    alreadydisabled = select_graphtype.disabled_options
    for element in alreadydisabled:
        disablegraphs.append(element)

    #add the new one
    value = select_graphtype.value
    select_graphtype.value = "Options"
    if value != "Options":
        disablegraphs.append(value)
    
    # set value
    select_graphtype.disabled_options = disablegraphs
    return

def show_graphtype_in_body(items , event):
    #unpack variables
    add_graphbox = items[0]
    select_graphtype = items[1]
    dashboard_body = items[2]
    graph_layout = items[3]

    if not event:
        return
    
    #add the selected graph type to body
    graphtype = event.new
    if graphtype == "Options":
        return 
    elif graphtype == "Scatter(1f)":
        #to keep the overview of the plots
        graph_layout.append("scatter with 1 feature")
        # the new body
        scatter1f = pn.Row(pn.layout.Divider(),
        pn.Column("### Create your scatter plot with 1 value", 
                pn.WidgetBox(input_title, autoinput_f1, select_group),
                btn_delete_f1
                ),
        pn.Column(fig_f1)
        )
        # append to body
        dashboard_body.append(scatter1f)
        dashboard_body.object = dashboard_body 
    elif graphtype == "Heatmap":
        #to keep the overview of the plots
        graph_layout.append("Heatmap")
        # the new body
        heatmap = pn.Row(pn.layout.Divider(),
        pn.Column("### Create your heatmap of the correlation between the features", 
                pn.WidgetBox(input_title_hm, multi_choice, checkbox_hm),
                btn_delete_hm   
                ),
        pn.Column(fig_hm)
        )
        # append to body
        dashboard_body.append(heatmap)
        dashboard_body.object = dashboard_body 
    elif graphtype == "Scatter(2f)":
        #to keep the overview of the plots
        graph_layout.append("scatter with 2 features")
        # the body
        scatter2f = pn.Row(pn.layout.Divider(),
            pn.Column( "### Create your scatter plot with 2 values",
                      pn.WidgetBox(input_title_f2, autoinput_f2_x, autoinput_f2_y, select_group_f2), 
                      btn_delete_f2
                      ),
            pn.Column(fig_f2)
        )
        #append to the body
        dashboard_body.append(scatter2f)
        dashboard_body.object = dashboard_body 
    elif graphtype == "Boxplot":
        #to keep the overview of the plots
        graph_layout.append("Boxplot")
        # the new body
        boxplot = pn.Row(pn.layout.Divider(),
            pn.Column("### Create your boxplots of the distribution from the features", 
                    pn.WidgetBox(multi_choice_bp),
                    btn_delete_bp 
                    ),
            pn.Column(fig_bp)
            )
        # append to body
        dashboard_body.append(boxplot)
        dashboard_body.object = dashboard_body 
        return

    #make invisible
    add_graphbox.visible = False
    graphtype_disabledoptions(select_graphtype)
        
    return

def read_inputfiles(file, radionmics = False):
    # transform the file to a df
    df = pd.read_csv(StringIO(file.decode("utf-8")))
    if radionmics:
        df.pop("Unnamed: 0")
    df.set_index("patient", inplace=True)

    return df


def file_radionomics(df_widget, event):
    file = event.new
    if file:
        df = read_inputfiles(file, radionmics = True)

        # edit the widget
        df_widget.value = df
        df_widget.visible = True
        return
    return 

def file_clinical(df_widget, event):
    file = event.new
    if file:
        df = read_inputfiles(file)

        # edit the widget
        df_widget.value = df
        df_widget.visible = True
        return 
    return 

def delete_graph(items, event):
    dashboard_body = items[0]
    graph_layout = items[1]

    locations = dict(enumerate(graph_layout))
    button = event.obj.description[26:]
    for loc, graphtype in locations.items():
        if graphtype == button:
            dashboard_body.pop(loc)
            graph_layout.remove(graphtype)

def select_features(dataframe_radio, event):
    #standard feature values
    feature_indices = {"shape features": 23, "first order features": 38, "GLCM features": 55,
                         "GLDM features": 79, "GLRLM features": 93, "GLZSM_features": 109, "NGTDM_features": 125}
    indices = [23 ,38 ,55, 79, 93, 109, 125]

    features = event.new
    df = dataframe_radio[0].value
    df_old = pd.DataFrame()

    for feature in features:
        indx = feature_indices[feature]
        if indx == 125:
            if not df_old.empty:
                df_new = df.iloc[:,indx:]
                df_old = pd.concat([df_old, df_new], axis=1 )
            else:
                df_old = df.iloc[:,indx:]
        else:
            if not df_old.empty:
                numb = [i for i,x in enumerate(indices) if x == indx]
                df_new = df.iloc[:, indx: indices[numb[0]+ 1] -1]
                df_old = pd.concat([df_old, df_new], axis=1)
            else:
                #aangegeven feature tot the volgende 
                numb = [i for i,x in enumerate(indices) if x == indx]
                df_old = df.iloc[:, indx: indices[numb[0]+ 1] -1]
    
    #change the col and index names
    ticks_names = list(df_old.columns)
    ticksnames = {}
    for names in ticks_names:
        ticksnames[names] = names.replace("_", " ")
    df_old.rename(index = ticksnames, columns= ticksnames, inplace=True)

    #make new dataframe visible
    dataframe_radio[1].value = df_old
    dataframe_radio[1].visible = True
    
    return df_old

def change_options(items, event):
    autoinput_f1 = items[0]
    autoinput_f2_x = items[1] 
    autoinput_f2_y = items[2]
    multi_choice = items[3]
    multi_choice_bp = items[4]

    dataframe = event.new

    autoinput_f1.options = list(dataframe.columns)
    autoinput_f2_x.options = list(dataframe.columns)
    autoinput_f2_y.options = list(dataframe.columns)
    multi_choice.options = list(dataframe.columns)
    multi_choice.value = list(dataframe.columns[0])
    multi_choice_bp.options = list(dataframe.columns)
    multi_choice_bp.value = list(dataframe.columns[0])

    return

def change_options_group(items, event):
    select_group = items[0]
    select_group_f2 = items[1] 

    dataframe = event.new
    df_nonan = dataframe.dropna(axis=1)
    select_group.options = list(df_nonan.columns)
    select_group_f2.options = list(df_nonan.columns)


dashboard = pn.template.BootstrapTemplate(
    title= "PlotRadionomics",
    header_background = "indigo",
    sidebar=[]
)

#Create widgets
## main
btn_add_graph = pn.widgets.Button(
    name="Add graph", 
    icon="plus", 
    button_type="primary", 
    description="Push to add a another graph", 
    visible=True
    )
select_graphtype = pn.widgets.Select(
    name="Choose your graph type", 
    options=["Options", "Scatter(1f)", "Scatter(2f)", "Boxplot", "Heatmap"], 
    disabled_options=[], 
    value="Options"
    )
radionomics_input = pn.widgets.FileInput(accept=".csv")
clinical_input = pn.widgets.FileInput(accept=".csv")
multi_choice_df = pn.widgets.MultiChoice(name='Choose the features you want to compare',
    options=["shape features", "first order features", "GLCM features",
              "GLDM features", "GLRLM features", "GLZSM features", "NGTDM features"])
pn_df_clinical = pn.widgets.DataFrame(temp_df ,visible = False,  reorderable= True )
pn_df_radionmics = pn.widgets.DataFrame(temp_df ,visible = False, reorderable= True)
pn_df_edit = pn.widgets.DataFrame(temp_df, visible= False, reorderable= True)


## Scatter(1f)
autoinput_f1 = pn.widgets.AutocompleteInput(
    name="Filter on feature", 
    options=options,
    case_sensitive=False, 
    search_strategy="includes",
    placeholder="Start to write feature"
    )
select_group = pn.widgets.Select(
    name="Based on wich group", 
    options=options_group,
    value = "Sex")
input_title = pn.widgets.TextInput(
    name="Graph title",
      placeholder="Enter a string here..."
      )
btn_delete_f1 = pn.widgets.Button(
    name="delete", 
    button_type="danger", 
    description="Push to delete the graph: scatter with 1 feature", 
)

## Heatmap
input_title_hm = pn.widgets.TextInput(
    name="Graph title",
      placeholder="Enter a string here..."
      )
multi_choice = pn.widgets.MultiChoice(name='Choose the features you want to compare',
    options=options,
    value = ["original shape Flatness"])
checkbox_hm = pn.widgets.Checkbox(name='All features')
btn_delete_hm = pn.widgets.Button(
    name="delete", 
    button_type="danger", 
    description="Push to delete the graph: Heatmap", 
)


## Scatter(2f)
autoinput_f2_x = pn.widgets.AutocompleteInput(
    name="Filter on feature for x-axis", 
    options=options,
    case_sensitive=False, 
    search_strategy="includes",
    placeholder="Start to write feature"
    )
autoinput_f2_y = pn.widgets.AutocompleteInput(
    name="Filter on feature for y-axis", 
    options=options,
    case_sensitive=False, 
    search_strategy="includes",
    placeholder="Start to write feature"
    )
select_group_f2 = pn.widgets.Select(
    name="Based on wich group", 
    options=options_group,
    value = "Sex"
    )
input_title_f2 = pn.widgets.TextInput(
    name="Graph title",
      placeholder="Enter a string here..."
      )
btn_delete_f2 = pn.widgets.Button(
    name="delete", 
    button_type="danger", 
    description="Push to delete the graph: scatter with 2 features", 
)

## boxplot
multi_choice_bp = pn.widgets.MultiChoice(name='Choose the features you want to see',
    options=options,
    value = ["original shape Flatness"])
btn_delete_bp = pn.widgets.Button(
    name="delete", 
    button_type="danger", 
    description="Push to delete the graph: Boxplot", 
)


#Bind widgets

## binding of the different figures with the widgets
fig_f1 = pn.bind(create_plot_f1, x=autoinput_f1, z=select_group, title=input_title, df_clini=pn_df_clinical, df_radionmics=pn_df_edit)
fig_f2 = pn.bind(create_plot_f2, x=autoinput_f2_x, y=autoinput_f2_y, z=select_group_f2, title=input_title,  df_clini=pn_df_clinical, df_radionmics=pn_df_edit)
fig_hm = pn.bind(create_plot_heat, title= input_title_hm, x=multi_choice, all=checkbox_hm,  df_radionmics=pn_df_edit)
fig_bp = pn.bind(create_boxplot,  df_shape=pn_df_edit, features=multi_choice_bp)

#Create body items
## contains the menu of what graph you want to select
add_graphbox = pn.WidgetBox(
    "## Create your graph", 
    select_graphtype, visible = False
    )

## welcome element
welcome = pn.Column(
    "# Welcome at PlotRadionomics",
    """Here you can make different plots of your radionomics data.
    For this you also need a clinical file with data of all the same patients in your radionomics.
    This data can be info you can filter the dataset on.
    Only csv files are accepted.""",
    pn.Row("Upload your file with the radionomics data here", radionomics_input,
    "Upload your file with the clinical patient data here", clinical_input),
    pn.Row(pn_df_radionmics, pn_df_clinical),
    pn.Row(multi_choice_df),
    pn.Row(pn_df_edit)
    )

#the body
body = pn.Column(pn.Column(
            welcome,
            "## Want to add a graph?", 
            btn_add_graph, add_graphbox
            ))

#list with all the graphtypes in the body
graph_layout = ["welcome"]

#watch the widgets
## add graph to the body
items = [add_graphbox, select_graphtype, body, graph_layout]
select_graphtype.link(items, callbacks={"value": show_graphtype_in_body})
## takes care of the graph select menu
items = [add_graphbox, select_graphtype, graph_layout]
btn_add_graph.link(items, callbacks={"value": add_graph_to_layout})
## takes care of the feature select menu
items = [pn_df_radionmics, pn_df_edit]
multi_choice_df.link(items, callbacks={"value": select_features})
## delete graphs of layout
items = [body, graph_layout]
btn_delete_f1.link(items, callbacks={"value": delete_graph})
btn_delete_f2.link(items, callbacks={"value": delete_graph})
btn_delete_hm.link(items, callbacks={"value": delete_graph})
## binding of the dataframes to the page
radionomics_input.link(pn_df_radionmics, callbacks={"value": file_radionomics})
clinical_input.link(pn_df_clinical, callbacks={"value": file_clinical})
## edit options in widgets
items = [select_group, select_group_f2]
pn_df_clinical.link(items, {"value": change_options_group})
items = [autoinput_f1, autoinput_f2_x, autoinput_f2_y, multi_choice, multi_choice_bp]
pn_df_edit.link(items, {"value": change_options})



dashboard.main.append(body)


dashboard.show();

Launching server at http://localhost:55176


         original shape Flatness  original shape LeastAxisLength  \
patient                                                            
4                       0.540468                       29.382158   
11                      0.536580                       20.921578   
12                      0.479827                       28.477824   
13                      0.481861                       22.561287   
16                      0.336308                        7.300546   
...                          ...                             ...   
596                     0.384389                       36.016806   
597                     0.415840                       25.254556   
598                     0.468391                       28.710600   
600                     0.201612                       13.332158   
602                     0.757112                       43.004789   

         original shape MajorAxisLength  \
patient                                   
4                            

als input file klopt dan pas mag er worden doorgegeven
input doorgeven aan bind objecten

1 of andere manier globals weghalen

heat en boxplot add

In [577]:
df_old = df_shape
#change the col and index names
ticks_names = list(df_old.columns)
ticksnames = {}
for names in ticks_names:
    ticksnames[names] = names.replace("_", " ")
df_old.rename(index = ticksnames, columns= ticksnames, inplace=True)

df_old

,original shape Flatness,original shape LeastAxisLength,original shape MajorAxisLength,original shape Maximum2DDiameterColumn,original shape Maximum2DDiameterRow,original shape Maximum2DDiameterSlice,original shape Maximum3DDiameter,original shape MeshVolume,original shape MinorAxisLength,original shape Sphericity,original shape SurfaceArea,original shape SurfaceVolumeRatio,original shape VoxelVolume,original firstorder 10Percentile
patient,,,,,,,,,,,,,,
4,0.540468,29.382158,54.364303,63.976558,61.294372,46.173586,64.876806,36047.000000,37.435671,0.604673,8727.224465,0.242107,36234,21.0
11,0.536580,20.921578,38.990604,44.721360,43.863424,33.120990,46.065171,14326.833333,26.340705,0.687436,4149.716086,0.289646,14404,14.1
12,0.479827,28.477824,59.350228,65.787537,66.850580,49.396356,68.161573,39800.833333,37.641266,0.608044,9271.372824,0.232944,39986,39.0
13,0.481861,22.561287,46.821157,52.430907,38.470768,45.221676,52.896125,18065.250000,30.897470,0.605112,5502.302457,0.304579,18150,2.0
16,0.336308,7.300546,21.707910,20.396078,23.086793,27.018512,27.018512,2032.166667,19.617722,0.644759,1203.352906,0.592153,2078,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.384389,36.016806,93.698856,86.764048,104.651804,63.505905,113.243101,84360.833333,52.907357,0.473039,19664.418897,0.233099,84614,-14.4
597,0.415840,25.254556,60.731494,62.609903,60.926185,40.804412,63.513778,22804.000000,33.599530,0.497842,7811.484381,0.342549,22916,21.0
598,0.468391,28.710600,61.296162,68.622154,69.339743,62.265560,72.166474,31999.166667,52.700886,0.414276,11765.755300,0.367689,32132,-94.5


In [568]:
indices = [2,4,6]
indx = 4
df = df_shape
numb = [i for i,x in enumerate(indices) if x == indx]
df_new = df_shape.iloc[:, indx: indices[numb[0]+ 1] -1]





,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_10Percentile,original_shape_Maximum2DDiameterRow
patient,,,,,,,,,,,,,,,
4,0.540468,29.382158,54.364303,63.976558,61.294372,46.173586,64.876806,36047.000000,37.435671,0.604673,8727.224465,0.242107,36234,21.0,61.294372
11,0.536580,20.921578,38.990604,44.721360,43.863424,33.120990,46.065171,14326.833333,26.340705,0.687436,4149.716086,0.289646,14404,14.1,43.863424
12,0.479827,28.477824,59.350228,65.787537,66.850580,49.396356,68.161573,39800.833333,37.641266,0.608044,9271.372824,0.232944,39986,39.0,66.850580
13,0.481861,22.561287,46.821157,52.430907,38.470768,45.221676,52.896125,18065.250000,30.897470,0.605112,5502.302457,0.304579,18150,2.0,38.470768
16,0.336308,7.300546,21.707910,20.396078,23.086793,27.018512,27.018512,2032.166667,19.617722,0.644759,1203.352906,0.592153,2078,22.0,23.086793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.384389,36.016806,93.698856,86.764048,104.651804,63.505905,113.243101,84360.833333,52.907357,0.473039,19664.418897,0.233099,84614,-14.4,104.651804
597,0.415840,25.254556,60.731494,62.609903,60.926185,40.804412,63.513778,22804.000000,33.599530,0.497842,7811.484381,0.342549,22916,21.0,60.926185
598,0.468391,28.710600,61.296162,68.622154,69.339743,62.265560,72.166474,31999.166667,52.700886,0.414276,11765.755300,0.367689,32132,-94.5,69.339743


In [544]:
def change_df(items, event):
    df = items[0]
    pn_df = items[1]
    feature = event.new

    pn_df.object = df[feature]
    
df = pn.widgets.DataFrame(df_shape, autosize_mode = "fit_viewport", reorderable= True)

multi_choice_temp = pn.widgets.MultiChoice(name='MultiSelect',
    options=['original_shape_Flatness', 'Banana', 'Pear', 'Strawberry'])


items = [df_shape, df]
multi_choice_temp.link(items, callbacks={"value", change_df})


pn.Column(multi_choice_temp, df)

ValueError: <function change_df at 0x0000026DDFCBAC00> parameter was not found in list of parameters of class MultiChoice

per tab werken om een nieuwe grafiek te maken
 op 1 pagina de mogelijkheid te geven om een extra ander soort grafiek te maken
 boxplot
 dus heatmap kan je verschillende factoren selecteren
 een grafiek met 2 verschillende features
 een grafiek waarmee je features per persoon laat zien

 mischien een bepaalde threshold toevoegen
